# 04. Свертки & сети

## План
0. Полносвязная сеть на MNIST (с прошлого семинара)
1. (DIY) Свертка
2. Сверточный слой
3. Сборка CNN
4. Обучение и результаты
5. (bonus) Различные способы организации слоев в pytorch

## 0. Пример с картинками: MNIST

Обучения на MNIST в курсе DL почти не избежать...

In [ ]:
import os
import glob
import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
#!pip install opencv-python
import cv2

MNIST - это ставший классикой датасет с изображениями рукописных цифр. На нем мы построим минимальный пример работы с изображениями.

In [ ]:
!wget https://github.com/myleott/mnist_png/raw/master/mnist_png.tar.gz
!tar -xzf mnist_png.tar.gz
!ls mnist_png/

В отличие от датасета, рассмотренного на прошлом семинаре, здесь мы будем передавать не непосредственно данные, а путь до папки с файлами; причем структуру мы считаем известной (`split/digit/*.png`).

In [ ]:
class MNISTDataset(Dataset):
    
    def __init__(self, root_dir):
        self.images_filenames = []
        self.class_labels = []
        for class_label in os.listdir(root_dir):
            for image_basename in os.listdir(os.path.join(root_dir, class_label)):
                if not image_basename.endswith(".png"):
                    continue
                image_filename = os.path.join(root_dir, class_label, image_basename)
                self.images_filenames.append(image_filename)
                self.class_labels.append(int(class_label))
    
    def __len__(self):
        return len(self.images_filenames)
    
    def __getitem__(self, i):
        image = cv2.imread(self.images_filenames[i], cv2.IMREAD_GRAYSCALE)
        label = self.class_labels[i]
        return image, label
    
    @staticmethod
    def collate_fn(items):
        images = np.zeros((len(items), 28*28), dtype=np.float32)
        labels = np.zeros(len(items), dtype=np.uint8)
        for i, (image, label) in enumerate(items):
            image = image / 255.
            images[i] = image.ravel()
            labels[i] = label
        return torch.tensor(images).float(), torch.tensor(labels).long()

**NB:** можно было сделать парсинг файлов на диске через `glob.glob()`

In [ ]:
train_dataset = MNISTDataset(root_dir="mnist_png/training")
len(train_dataset)

Посмотрим на сами данные из датасета:

In [ ]:
image, label = train_dataset[0]
plt.imshow(image, cmap="gray")
plt.show()

In [ ]:
plt.imshow(image[:14, -14:], cmap="gray")

Вспомогательная функция для массовой визуализации:

In [ ]:
def show_images_with_captions(images, captions=None, ncol=8):
    nrow = len(images) // ncol
    
    plt.figure(figsize=(16, 16 * nrow // ncol))
    for i in range(len(images)):
        plt.subplot(nrow, ncol, i + 1)
        plt.imshow(images[i], cmap="gray")
        if captions is not None:
            plt.title(captions[i])
        plt.grid(False)
        plt.axis(False)
    plt.show()

In [ ]:
sample_indices = np.random.choice(len(train_dataset), size=64, replace=False)

sample_images = []
sample_captions = []
for i in sample_indices:
    image, label = train_dataset[i]
    sample_images.append(image)
    sample_captions.append(f"gt: {label}")

In [ ]:
show_images_with_captions(sample_images, sample_captions)

Зарядим теперь обучение сети чуть глубже (3 слоя), да еще и с BatchNorm1d:

In [ ]:
num_epochs = 10
batch_size = 32
lr = 3e-4

# device = torch.device("cpu")
device = torch.device("cuda:0")

In [ ]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=train_dataset.collate_fn
)

val_dataset = MNISTDataset(root_dir="mnist_png/testing/")
val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=train_dataset.collate_fn
)

In [ ]:
from torch.nn import Sequential, Linear, BatchNorm1d, ReLU

In [ ]:
model = Sequential(
    Linear(28*28, 512),
    ReLU(inplace=True),
    BatchNorm1d(512),
    Linear(512, 1024),
    ReLU(inplace=True),
    BatchNorm1d(1024),
    Linear(1024, 10)
)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Лосс:

In [ ]:
from torch.nn import CrossEntropyLoss
loss_fn = CrossEntropyLoss()

Функции для обучения / валидации:

In [ ]:
def train_epoch(model, dataloader, optimizer, loss_fn, epoch, device=device):
    model.train()
    model = model.to(device)
    
    losses = []
    for batch in dataloader:
        xs, ys_true = batch
                
        ys_pred = model(xs.to(device))
        loss = loss_fn(ys_pred, ys_true.to(device))
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
                
        losses.append(loss.cpu().item())
    
    return np.mean(losses)


def val_epoch(model, dataloader, loss_fn, device=device):
    model.eval()
    
    losses = []
    preds = []
    for batch in dataloader:
        xs, ys_true = batch
        with torch.no_grad():
            ys_pred = model(xs.to(device))
        
        loss = loss_fn(ys_pred, ys_true.to(device))        
        losses.append(loss.item())
        
        preds.append(ys_pred.cpu().numpy())
    
    preds = np.concatenate(preds, axis=0)
    return np.mean(losses), preds

In [ ]:
losses = []
val_losses = []
val_preds = []
for epoch in tqdm.trange(num_epochs):
    loss = train_epoch(model, train_dataloader, optimizer, loss_fn, epoch, device)
    losses.append(loss)
    
    val_loss, preds = val_epoch(model, val_dataloader, loss_fn)
    val_losses.append(val_loss)
    val_preds.append(preds)

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(losses, label="train")
plt.plot(val_losses, label="val")
plt.xlabel("epoch")
plt.ylabel("xEntLoss")
plt.legend()
plt.grid()
plt.show()

Соберем все предсказания / gt-лейблы, чтобы посчитать метрику Accuracy и сделать визуализацию:

In [ ]:
val_pred_labels = []
for val_pred in val_preds[-1]:
    pred_label = np.argmax(val_pred)
    val_pred_labels.append(pred_label)
val_pred_labels = np.asarray(val_pred_labels)

In [ ]:
val_labels = []
for image, label in val_dataset:
    val_labels.append(label)
val_labels = np.asarray(val_labels)

In [ ]:
acc = (val_pred_labels == val_labels).mean()
acc

Посмотрим, как соотносятся истинные лейблы и предсказанные моделью:

In [ ]:
sample_indices = np.random.choice(len(val_dataset), size=64, replace=False)

sample_images = []
sample_captions = []
for i in sample_indices:
    image, label = val_dataset[i]
    pred_label = val_pred_labels[i]
    sample_images.append(image)
    sample_captions.append(f"gt: {label} | pred: {pred_label}")

In [ ]:
show_images_with_captions(sample_images, sample_captions)

Можем отдельно отрисовать те примеры из валидации, на которых модель ошибается:

In [ ]:
sample_indices = np.random.choice(np.where(val_labels != val_pred_labels)[0], size=64, replace=False)

sample_images = []
sample_captions = []
for i in sample_indices:
    image, label = val_dataset[i]
    pred_label = val_pred_labels[i]
    sample_images.append(image)
    sample_captions.append(f"gt: {label} | pred: {pred_label}")

In [ ]:
show_images_with_captions(sample_images, sample_captions)

In [ ]:
print((val_labels != val_pred_labels).sum())

## 1. Свертка

### 1.1. (DIY) Одномерная свертка

В этом блоке предлагается самостоятельно реализовать механизм одномерной свертки с поддержкой добавления паддинга.

**Задача**: реализовать функцию для добавления паддингов тремя способами:
* zero
* replicate
* reflect

In [ ]:
def pad_1d(signal, size, kind):
    
    # YOUR CODE HERE
    
    # signal_padded = ...
    
    # if kind == "zero":
        # pass
    # elif kind == "replicate":
        # pass
    # elif kind == "reflect":
        # pass
    # else:
        # raise NotImplementedError(kind)
        
    # END OF YOUR CODE
    
    return signal_padded

In [ ]:
signal = np.arange(10)
signal

In [ ]:
np.testing.assert_array_equal(pad_1d(signal, 2, "zero"), np.array([0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 0]))

In [ ]:
np.testing.assert_array_equal(pad_1d(signal, 2, "replicate"), np.array([0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 9]))

In [ ]:
np.testing.assert_array_equal(pad_1d(signal, 2, "reflect"), np.array([2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 8, 7]))

In [ ]:
np.testing.assert_array_equal(pad_1d(signal, 0, "zero"), np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

**Задача**: реализовать (брутфорс) функцию вычисления свертки в одномерном случае с предварительным паддингом. 

In [ ]:
def convolve_1d(signal, kernel, pad_size, pad_kind):
    
    signal_padded = pad_1d(signal, pad_size, pad_kind)
    
    # YOUR CODE HERE
    
    # signal_padded_convolved = ...
    
    # END OF YOUR CODE
    
    return signal_padded_convolved

In [ ]:
signal = np.arange(10)
signal

In [ ]:
kernel = np.asarray([0, 0, 0])
for pad_kind in ("zero", "replicate", "reflect"):
    np.testing.assert_array_equal(convolve_1d(signal, kernel, 1, pad_kind), np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [ ]:
kernel = np.asarray([0, 1, 0])
for pad_kind in ("zero", "replicate", "reflect"):
    np.testing.assert_array_equal(convolve_1d(signal, kernel, 1, pad_kind), np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

Можно использовать полученную функцию свертки для фильтрации шумного одномерного сигнала:

In [ ]:
eps = 1e-6
def sinc(x):
    return (np.sin(x) + eps) / (x + eps)

In [ ]:
xs = np.linspace(0, 20, 1000)
signal = sinc(xs)
noise = np.random.normal(size=len(signal)) / 10
signal += noise

plt.figure(figsize=(12, 5))
plt.plot(signal)
plt.grid(True)
plt.show()

In [ ]:
k = 51
p = k // 2
kernel = np.ones(k, dtype=np.float32) / k
signal_smoothed = convolve_1d(signal, kernel, p, "replicate")

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(signal, label="signal")
plt.plot(signal_smoothed, label=f"smoothed, k={k}")
plt.grid(True)
plt.legend()
plt.show()

### 1.2. Двумерная свертка

Для демонстраций нам понадобится какая-нибудь картинка, скачаем ее (или любую другую):

In [ ]:
!wget -O volleyball.jpeg https://img.olympicchannel.com/images/image/private/t_social_share_thumb/f_auto/primary/bwkrhsijg1gb4oxwc7zf

In [ ]:
image = cv2.imread("volleyball.jpeg", cv2.IMREAD_GRAYSCALE)
image.shape, image.dtype

In [ ]:
plt.figure(figsize=(12, 5))
plt.imshow(image, cmap="gray")

In [ ]:
image = image.astype(np.float32) / 255.

In [ ]:
plt.figure(figsize=(12, 5))
plt.imshow(image, cmap="gray")

В OpenCV есть реализованный механизм свертки с заданным ядром. Используем его для того, чтобы применить к картинке операцию сглаживания ([box filter](https://en.wikipedia.org/wiki/Box_blur)):

In [ ]:
k = 1

kernel = np.ones((k, k), dtype=np.float32) / (k * k)

In [ ]:
image_conved = cv2.filter2D(image, -1, kernel)

In [ ]:
plt.figure(figsize=(12, 5))
plt.imshow(image_conved, cmap="gray")

## 2. Сверточный слой

OpenCV предоставляет функцию для свертки с заданным ядром. Но нам-то нужны обучаемые свертки!

In [ ]:
from torch.nn import Conv2d

In [ ]:
conv = Conv2d(
    in_channels=1,
    out_channels=1,
    kernel_size=5,
    padding=2
)

conv

In [ ]:
conv.weight.shape  # == out_channels, in_channels, k, k

Очень хочется применить этот слой к нашему изображению:

In [ ]:
conv(image)

Конечно же, чтобы пользоваться методами и классами из pytorch, надо обернуть данные в `torch.Tensor`.

* **NB**: помним о том, что картинки размером (h, w, 1) `matplotlib` не поймет - либо "серые" (h, w), либо цветные (h, w, 3) (либо вообще (h, w, 4), если с прозрачностью).

In [ ]:
def image_to_tensor(image):
    if image.ndim == 2:
        image = image[:, :, np.newaxis]
    if image.dtype == np.uint8:
        image = image.astype(np.float32) / 255.
        
    tensor = torch.from_numpy(image)
    tensor = tensor.permute(2, 0, 1).unsqueeze(0)
    
    return tensor


def tensor_to_image(tensor):
    image = tensor[0].permute(1, 2, 0).numpy()
    if image.shape[-1] == 1:
        image = image[:, :, 0]
    return image

In [ ]:
tensor = image_to_tensor(image)
tensor.shape

Теперь можно отправить тензор с нашим изображением в слой:

In [ ]:
y = conv(tensor)
y

In [ ]:
y.shape

In [ ]:
image_conved = tensor_to_image(y.detach())

plt.figure(figsize=(12, 5))
plt.imshow(image_conved, cmap="gray")

Повторим наш трюк с реализацией `box filter`, но теперь "записав" наше ядро в веса сверточного слоя `Conv2d`:

In [ ]:
k = 21
p = k // 2

conv = Conv2d(
    in_channels=1,
    out_channels=1,
    kernel_size=k,
    padding=p
)
conv.requires_grad_(False)

In [ ]:
conv.weight[0] = 1 / (k * k)

In [ ]:
y = conv(tensor)

plt.figure(figsize=(12, 5))
plt.imshow(tensor_to_image(y), cmap="gray")
plt.show()

Сверткам из 1 канала в 1 сыт не будешь - в современных моделях характерные глубины тензоров ~ 32, 64, 128, 256, 512, 1024, ...

In [ ]:
image = cv2.imread("volleyball.jpeg")
image.shape, image.dtype

In [ ]:
plt.figure(figsize=(12, 5))
plt.imshow(image)

Неприятная вещь в `opencv` - по умолчанию изображения считываются в формате BGR вместо RGB. `matplotlib` к такому не готов и ждет изображения в RGB. Для этого придется делать конвертацию вручную:

In [ ]:
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(12, 5))
plt.imshow(image)

[*BGR was a choice made for historical reasons and now we have to live with it. In other words, BGR is the horse’s ass in OpenCV.*](https://learnopencv.com/why-does-opencv-use-bgr-color-format/)

Теперь у нашего тензора будет 3 канала, надо учесть это при создании сверточного слоя:

In [ ]:
conv = Conv2d(
    in_channels=3,
    out_channels=16,
    kernel_size=5,
    padding=2
)

conv

In [ ]:
conv.weight.shape

In [ ]:
x = image_to_tensor(image)
x.shape

In [ ]:
y = conv(x)
y.shape

**Задание**: реализуйте сверточный слой (со сверткой размера 1х1), который изменит порядок следования каналов в тензоре глубины 3 на противоположный.

In [ ]:
def create_channels_permutator():
    
    # YOUR CODE HERE

    # conv = 

    # END OF YOUR CODE

    return conv

In [ ]:
x = image_to_tensor(image)

In [ ]:
permutator = create_channels_permutator()
with torch.no_grad():
    x_permuted = permutator(x)

for i in range(3):
    np.testing.assert_array_equal(x[0, i].numpy(), x_permuted[0, -i-1].numpy())

In [ ]:
plt.figure(figsize=(12, 5))
plt.imshow(tensor_to_image(x_permuted))

## 3. Сборка CNN

Теперь, когда мы познакомились со сверточными слоями, пришла пора собрать нейросеть на их основе.

In [ ]:
from torch.nn import Conv2d, MaxPool2d, AvgPool2d

Кроме сверточных слоев нам пригодятся пулинги:

In [ ]:
x = torch.randn(1, 3, 32, 32)

In [ ]:
max_pool2d = MaxPool2d((2, 2))
max_pool2d(x).shape

In [ ]:
avg_pool2d = AvgPool2d((2, 2))
avg_pool2d(x).shape

И еще два важных слоя:
* `BatchNorm2d` для батч-нормализации
* `Flatten` для "вытягивания" тензора в вектор (для финальной классификации)

In [ ]:
from torch.nn import BatchNorm2d, Flatten

In [ ]:
bn = BatchNorm2d(3)
bn(x).shape

In [ ]:
flatten = Flatten()
flatten(x).shape

Собираем (с параметрами для датасета MNIST):

In [ ]:
my_cnn = Sequential(            # b x 1 x 28 x 28
    Conv2d(1, 8, (3, 3), 1, 1), # b x 8 x 28 x 28
    BatchNorm2d(8),             # ...
    ReLU(inplace=True),         # ...
    MaxPool2d((2, 2)),          # b x 8 x 14 x 14

    Conv2d(8, 32, (3, 3), 1, 1),# b x 32 x 14 x 14
    BatchNorm2d(32),            # ...
    ReLU(inplace=True),         # ...
    MaxPool2d((2, 2)),          # b x 32 x 7 x 7

    Conv2d(32, 64, (3, 3), 1, 1),   # b x 64 x 7 x 7
    ReLU(inplace=True),         # ...

    AvgPool2d((7, 7)),          # b x 64 x 1 x 1

    Flatten(),                  # b x 64
    Linear(64, 10)              # b x 10
)

In [ ]:
my_cnn

In [ ]:
x = torch.randn(4, 1, 28, 28)

In [ ]:
my_cnn(x).shape

## 4. Обучение

In [ ]:
optimizer = torch.optim.Adam(my_cnn.parameters(), lr=lr)

Лосс:

In [ ]:
from torch.nn import CrossEntropyLoss
loss_fn = CrossEntropyLoss()

In [ ]:
def collate_fn(items):
    images = np.zeros((len(items), 1, 28, 28), dtype=np.float32)
    labels = np.zeros(len(items), dtype=np.uint8)
    for i, (image, label) in enumerate(items):
        image = image / 255.
        images[i] = image
        labels[i] = label
    return torch.tensor(images).float(), torch.tensor(labels).long()

In [ ]:
val_dataset = MNISTDataset(root_dir="mnist_png/training/")
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn
)

val_dataset = MNISTDataset(root_dir="mnist_png/testing/")
val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=collate_fn
)

In [ ]:
losses = []
val_losses = []
val_preds = []
for epoch in tqdm.trange(num_epochs):
    loss = train_epoch(my_cnn, train_dataloader, optimizer, loss_fn, epoch, device)
    losses.append(loss)
    
    val_loss, preds = val_epoch(my_cnn, val_dataloader, loss_fn, device)
    val_losses.append(val_loss)
    val_preds.append(preds)

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(losses, label="train")
plt.plot(val_losses, label="val")
plt.xlabel("epoch")
plt.ylabel("xEntLoss")
plt.legend()
plt.grid()
plt.show()

In [ ]:
val_pred_labels = []
for val_pred in val_preds[-1]:
    pred_label = np.argmax(val_pred)
    val_pred_labels.append(pred_label)
val_pred_labels = np.asarray(val_pred_labels)

In [ ]:
val_labels = []
for image, label in val_dataset:
    val_labels.append(label)
val_labels = np.asarray(val_labels)

In [ ]:
acc = (val_pred_labels == val_labels).mean()
acc

In [ ]:
sample_indices = np.random.choice(len(val_dataset), size=64, replace=False)

sample_images = []
sample_captions = []
for i in sample_indices:
    image, label = val_dataset[i]
    pred_label = val_pred_labels[i]
    sample_images.append(image)
    sample_captions.append(f"gt: {label} | pred: {pred_label}")

In [ ]:
show_images_with_captions(sample_images, sample_captions)

Можем отдельно отрисовать те примеры из валидации, на которых модель ошибается:

In [ ]:
sample_indices = np.random.choice(np.where(val_labels != val_pred_labels)[0], size=64, replace=False)

sample_images = []
sample_captions = []
for i in sample_indices:
    image, label = val_dataset[i]
    pred_label = val_pred_labels[i]
    sample_images.append(image)
    sample_captions.append(f"gt: {label} | pred: {pred_label}")

In [ ]:
show_images_with_captions(sample_images, sample_captions)

In [ ]:
print((val_labels != val_pred_labels).sum())

Посмотрим, как обстоит дело с устойчивостью моделей, например, к смещениям:

In [ ]:
def shift_image(image, min_shift=3, max_shift=6):
    shift = np.random.randint(min_shift, max_shift+1)

    output = np.zeros_like(image)
    p = np.random.uniform()
    pad_left = pad_right = pad_top = pad_bottom = 0
    
    if p <= 0.25:  # <-
        output[:, :-shift] = image[:, shift:]
    elif p < 0.5:  # ->
        output[:, shift:] = image[:, :-shift]
    elif p < 0.75: # ^
        output[:-shift, :] = image[shift:, :]
    else:          # v
        output[shift:, :] = image[:-shift, :]
    
    return output

In [ ]:
# i = np.random.choice(np.where(val_labels == val_pred_labels)[0], size=1)[0]
i = np.random.choice(len(val_dataset), size=1)[0]

image = val_dataset[i][0]
plt.imshow(image)

In [ ]:
plt.imshow(shift_image(image))

In [ ]:
model.eval();
my_cnn.eval();

for j in range(10):
    tensor = image_to_tensor(shift_image(image)).to(device)

    with torch.no_grad():
        preds_fcn = model(tensor.view(1, -1))
    label_fcn = preds_fcn.cpu().numpy().ravel().argmax()
    
    
    with torch.no_grad():
        preds_cnn = my_cnn(tensor)
    label_cnn = preds_cnn.cpu().numpy().ravel().argmax()

    print(label_fcn, label_cnn)

Как видим, природа сверточного слоя делает его более устойчивым к смещению (translation invariance), что и делает его таким полезным при работе с изображениями (и другими "вытянутыми" сигналами).

## 5. (bonus) Различные способы организации слоев в pytorch


In [ ]:
from torch import nn

In [ ]:
x = torch.randn(1, 1, 32, 32)

In [ ]:
my_cnn_seq = Sequential(        # b x 1 x 28 x 28
    Conv2d(1, 8, (3, 3), 1, 1), # b x 8 x 28 x 28
    ReLU(inplace=True),         # ...
    BatchNorm2d(8),             # ...
    MaxPool2d((2, 2)),          # b x 8 x 14 x 14
    Conv2d(8, 32, (3, 3), 1, 1),# b x 32 x 14 x 14   
    ReLU(inplace=True),         # ...
    BatchNorm2d(32),            # ...
    MaxPool2d((2, 2)),          # b x 32 x 7 x 7
    Conv2d(32, 64, (3, 3), 1, 1),   # b x 64 x 7 x 7
    ReLU(inplace=True),         # ...
    AvgPool2d((7, 7)),          # b x 64 x 1 x 1
    Flatten(),                  # b x 64
    Linear(64, 10)              # b x 10
)

In [ ]:
class MyCNNBlock(nn.Module):
    def __init__(self, in_features, out_features, relu=True, bn=True, maxpool=True):
        super(MyCNNBlock, self).__init__()

        self.conv = Conv2d(in_features, out_features, (3, 3), 1, 1)
        self.relu = ReLU(inplace=True) if relu else nn.Identity()
        self.bn = BatchNorm2d(out_features) if bn else nn.Identity()
        self.maxpool = MaxPool2d((2, 2)) if maxpool else nn.Identity()

    def forward(self, x):
        y1 = self.conv(x)
        y2 = self.relu(y1)
        y3 = self.bn(y2)
        y4 = self.maxpool(y3)
        return y4

In [ ]:
block = MyCNNBlock(1, 8)
block(x).shape

* Больше гибкости в работе с промежуточными значениями
* Можно делать "непрямое" течение тензоров (ResNet)
* Проще эксперименты

In [ ]:
my_cnn_blocked = Sequential(
    MyCNNBlock(1, 8),
    MyCNNBlock(8, 32),
    MyCNNBlock(32, 64, bn=False, maxpool=False),
    AvgPool2d((7, 7)),          # b x 64 x 1 x 1
    Flatten(),                  # b x 64
    Linear(64, 10)              # b x 10
)

In [ ]:
my_cnn_blocked(x)